# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
pip install cartopy geoviews pyproj


Note: you may need to restart the kernel to use updated packages.


In [2]:
conda install -c conda-forge cartopy geoviews pyproj


error: incomplete escape \U at position 28

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aasiaat,68.7098,-52.8699,37.87,90,100,9.78,GL,1724890740
1,1,agdas,40.6500,47.4761,73.87,46,0,2.33,AZ,1724890783
2,2,kapa'a,22.0752,-159.3190,82.72,70,75,9.22,US,1724890739
3,3,kingscliff,-28.2667,153.5667,81.95,69,0,13.80,AU,1724890783
4,4,iskateley,67.6803,53.1512,45.36,91,77,12.95,RU,1724890784


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=1.0,
    color="City"
)

# Display the map
# YOUR CODE HERE
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
weather_clean_df = city_data_df.dropna(how='any')
ideal_weather_df = weather_clean_df.loc[
    (weather_clean_df['Wind Speed'] < 8.0) & 
    (weather_clean_df['Max Temp'] < 86) & 
    (weather_clean_df['Humidity'] < 40)
]
# Drop any rows with null values
# YOUR CODE HERE
# Display sample data
# YOUR CODE HERE
ideal_weather_df.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
36,36,vilhena,-12.7406,-60.1458,80.82,39,11,3.44,BR,1724890686
69,69,campo verde,-20.4167,-54.0667,73.31,30,61,6.15,BR,1724890793
148,148,lucapa,-8.4192,20.7447,71.55,32,8,2.51,AO,1724890805
150,150,mokrous,51.2333,47.5167,67.21,38,74,1.01,RU,1724890806
151,151,alice springs,-23.7000,133.8833,80.15,24,5,1.14,AU,1724890721
174,174,sanctuary point,-35.1114,150.6290,63.88,29,47,5.08,AU,1724890809
180,180,apolinario saravia,-24.4328,-63.9954,64.58,34,38,3.33,AR,1724890810
255,255,el bayadh,33.6832,1.0193,79.47,25,23,4.79,DZ,1724890822
258,258,dzhangala,49.2167,50.3333,67.82,35,3,6.26,KZ,1724890823
289,289,bouarfa,32.5300,-1.9500,78.37,35,52,6.24,MA,1724890828


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_weather_df[['City', 'Country', 'Lat','Lng','Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ''
# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
36,vilhena,BR,-12.7406,-60.1458,39,
69,campo verde,BR,-20.4167,-54.0667,30,
148,lucapa,AO,-8.4192,20.7447,32,
150,mokrous,RU,51.2333,47.5167,38,
151,alice springs,AU,-23.7000,133.8833,24,
174,sanctuary point,AU,-35.1114,150.6290,29,
180,apolinario saravia,AR,-24.4328,-63.9954,34,
255,el bayadh,DZ,33.6832,1.0193,25,
258,dzhangala,KZ,49.2167,50.3333,35,
289,bouarfa,MA,32.5300,-1.9500,35,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:

# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key,
    'limit': 10
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"  # Note: lng, lat order is important in some APIs
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    response_json = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head(20)

Starting hotel search
vilhena - nearest hotel: No hotel found
campo verde - nearest hotel: No hotel found
lucapa - nearest hotel: No hotel found
mokrous - nearest hotel: No hotel found
alice springs - nearest hotel: Aurora Alice Springs
sanctuary point - nearest hotel: Dungowan Accommodation
apolinario saravia - nearest hotel: No hotel found
el bayadh - nearest hotel: No hotel found
dzhangala - nearest hotel: Жаңақала ЛПУ жатақханасы
bouarfa - nearest hotel: Climat du Maroc مناخ المغرب
sao vicente - nearest hotel: Pousada Vitória
meadow lake - nearest hotel: No hotel found
lago da pedra - nearest hotel: No hotel found
vernon - nearest hotel: Okanagan Royal Park Inn
mbuji-mayi - nearest hotel: Hôtel Gloria
feijo - nearest hotel: Hotel JOAFRAN
ulladulla - nearest hotel: Hotel Marlin
chongwe - nearest hotel: No hotel found
torghay - nearest hotel: No hotel found
campo gallo - nearest hotel: Hotel La Plaza


,City,Country,Lat,Lng,Humidity,Hotel Name
36,vilhena,BR,-12.7406,-60.1458,39,No hotel found
69,campo verde,BR,-20.4167,-54.0667,30,No hotel found
148,lucapa,AO,-8.4192,20.7447,32,No hotel found
150,mokrous,RU,51.2333,47.5167,38,No hotel found
151,alice springs,AU,-23.7000,133.8833,24,Aurora Alice Springs
174,sanctuary point,AU,-35.1114,150.6290,29,Dungowan Accommodation
180,apolinario saravia,AR,-24.4328,-63.9954,34,No hotel found
255,el bayadh,DZ,33.6832,1.0193,25,No hotel found
258,dzhangala,KZ,49.2167,50.3333,35,Жаңақала ЛПУ жатақханасы
289,bouarfa,MA,32.5300,-1.9500,35,Climat du Maroc مناخ المغرب


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)